# Analyse des Anomalies de Précipitations : Identifier les Périodes de Sécheresse


Le fichier de données utilisé dans cette analyse provient du site data.gouv.fr et contient des informations météorologiques quotidiennes enregistrées par divers postes météorologiques en France. Les données incluent des informations sur les précipitations, les températures minimales et maximales, la vitesse du vent, et d'autres paramètres météorologiques. L'URL source est https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz.


L'objectif de cette analyse est d'identifier les périodes de sécheresse anormales en agrégeant les données météorologiques mensuelles. Pour cela, nous allons étudier l'évolution des précipitations sur une période donnée et identifier les mois où les précipitations sont inférieures à la moyenne globale.


## Méthodologie


La méthodologie utilisée dans cette analyse consiste à convertir la date en format mensuel, agréger les précipitations mensuelles, et calculer la moyenne mensuelle des précipitations sur toute la période. Ensuite, nous sélectionnons les mois où les précipitations sont inférieures à la moyenne globale et calculons l'anomalie de précipitation pour ces mois. Les résultats sont ensuite visualisés à l'aide de graphiques pour mettre en évidence les périodes de sécheresse anormales.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH 
-- Conversion de la date et agrégation mensuelle des précipitations
monthly_precipitation AS (
  SELECT 
    CAST(SUBSTRING("AAAAMMJJ", 1, 6) AS INTEGER) AS date_mois,
    AVG(CAST("RR" AS DOUBLE)) AS avg_rr
  FROM 
    loaded_dataset
  WHERE 
    "RR" IS NOT NULL AND "QRR" = '1'
  GROUP BY 
    CAST(SUBSTRING("AAAAMMJJ", 1, 6) AS INTEGER)
),
-- Calcul de la moyenne mensuelle des précipitations sur toute la période
monthly_avg_precipitation AS (
  SELECT 
    date_mois,
    avg_rr,
    AVG(avg_rr) OVER () AS global_avg_rr
  FROM 
    monthly_precipitation
)
-- Sélection des mois où les précipitations sont inférieures à la moyenne globale
SELECT 
  date_mois,
  avg_rr,
  global_avg_rr,
  (avg_rr - global_avg_rr) / global_avg_rr AS anomaly_precipitation
FROM 
  monthly_avg_precipitation
WHERE 
  avg_rr < global_avg_rr
ORDER BY 
  date_mois """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 500 lignes


,date_mois,avg_rr,global_avg_rr,anomaly_precipitation
0,195001,1.885694,3.792464,-0.502779
1,195003,2.957083,3.792464,-0.220274
2,195005,3.029635,3.792464,-0.201143
3,195006,2.012319,3.792464,-0.469390
4,195007,0.623900,3.792464,-0.835489


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, une librairie de visualisation de données interactive et flexible. Cette technologie est adaptée car elle permet de créer des graphiques complexes et interactifs, facilitant l'exploration et la compréhension des données représentées. Plotly offre une grande personnalisation et convient parfaitement à la visualisation de séries temporelles comme dans cet exemple.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df['date_mois'] = pd.to_datetime(df['date_mois'].astype(str), format='%Y%m')
df = df.sort_values('date_mois')

for win in [6, 12]:
    df[f'nomal_anom_{win}m'] = df['anomaly_precipitation'].rolling(window=win).mean()

dataviz = make_subplots(specs=[[{"secondary_y": True}]])

dataviz.add_trace(
    go.Scatter(
        x=df['date_mois'],
        y=df['anomaly_precipitation'],
        mode='lines',
        name='Anomalie brute',
        line=dict(color='#c4c4c4', width=1),
        hovertemplate='%{y:.3f}<extra>Anomalie brute</extra>'
    ),
    secondary_y=False
)

dataviz.add_trace(
    go.Scatter(
        x=df['date_mois'],
        y=df['nomal_anom_12m'],
        mode='lines',
        name='12 mois lissés',
        line=dict(color='#3f72af', width=3),
        hovertemplate='%{y:.3f}<extra>12 mois lissés</extra>'
    ),
    secondary_y=False
)

dataviz.add_trace(
    go.Scatter(
        x=df['date_mois'],
        y=df['nomal_anom_6m'],
        mode='lines',
        name='6 mois lissés',
        line=dict(color='#0fd0ff', width=2.5),
        hovertemplate='%{y:.3f}<extra>6 mois lissés</extra>'
    ),
    secondary_y=False
)

dataviz.add_trace(
    go.Bar(
        x=df['date_mois'],
        y=df['anomaly_precipitation'],
        name='Ampleur anomalie',
        marker=dict(
            color=['#c41e3a' if v < 0 else '#00a86b' for v in df['anomaly_precipitation']],
            opacity=0.15
        ),
        hovertemplate='%{y:.3f}<extra>Ampleur</extra>'
    ),
    secondary_y=True
)

dataviz.update_layout(
    title=dict(
        text="Mets un peu de lissage - Évolution des anomalies de précipitations [Climat Sécheresse]",
        font=dict(family='Arial, sans-serif', size=16)
    ),
    xaxis=dict(
        title='Date',
        showgrid=True,
        gridwidth=1,
        gridcolor='#ececec'
    ),
    yaxis=dict(
        title='Anomalie de précipitation',
        showgrid=True,
        gridwidth=1,
        gridcolor='#ececec',
        rangemode='tozero'
    ),
    margin=dict(l=20, r=20, t=50, b=20),
    plot_bgcolor='white',
    paper_bgcolor='white',
    hovermode='x unified',
    showlegend=True
)

dataviz.update_yaxes(title_text="Ampleur anomalie", secondary_y=True, showgrid=False, rangemode='tozero')
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250803_101615.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250803_101615.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250803_101615.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250803_101615.png


--> Image Plotly sauvegardée avec succès.
